In [2]:
import math
import hashlib
import time
import random
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

prime_list = set([2, 3, 5])
def is_prime(n):
    if n in prime_list:
        return True
    root = math.floor(math.sqrt(n))
    for prime in filter(lambda x: x <= root, prime_list):
        if n % prime == 0:
            return False
    for i in range(max(prime_list) + 1, root):
        if is_prime(i):
            # print(i)
            prime_list.add(i)
        if n % i == 0:
            return False
    return True

import math

def discrete_logarithm(g, p):
    m = math.ceil(math.sqrt(p - 1))
    baby_steps = {}
    val = 1
    for j in range(m):
        baby_steps[val] = j
        val = (val * g) % p
    g_inv_m = pow(g, p - 1 - m, p)
    val = pow(g, m, p)
    for i in range(m):
        if val in baby_steps:
            return i * m + baby_steps[val]
        val = (val * g_inv_m) % p
    return None

def get_divisors(n):
    divisors = []
    root = math.floor(math.sqrt(n))
    i = 2
    while n > 1 and i <= n:
        if is_prime(n):
            divisors.append(n)
            break
        if (n % i == 0):
            divisors.append(i)
            while n % i == 0:
                n //= i
                print(n)
        i += 1
    return divisors

In [38]:
p, q = 13, 17
n, phi = 221, 192
e, d = 17, 113

def hash_to_float(hash_digest):
    sum = 0

class RSA_ES:
    @staticmethod
    def sign(message, d, n):
        start = time.time()
        hash = int(hashlib.md5(message.encode()).hexdigest(), 16)
        signed_message = pow(hash, d, n)
        print(f'RSA sign time: {time.time() - start}')
        return signed_message
    @staticmethod
    def verify(message, S, e, n):
        start = time.time()
        real_hash = int(hashlib.md5(message.encode()).hexdigest(), 16) % n
        got_hash = pow(S, e, n)
        print(f'RSA verify time: {time.time() - start}')
        return real_hash == got_hash
    
signed = RSA_ES.sign('m', d, n)
print(RSA_ES.verify('m', signed, e, n))

RSA sign time: 0.0
RSA verify time: 0.0
True


In [37]:
class EL_GAMAL_ES:
    @staticmethod
    def generate_keys(p, g):
        private_key = EL_GAMAL_ES._get_random_range(1, p - 1)
        public_key = pow(g, private_key, p)
        return private_key, public_key
    
    @staticmethod
    def sign(message, p, g, private_key):
        start = time.time()
        k = p - 1
        while k % (p - 1) == 0:
            k = EL_GAMAL_ES._get_random_range(1, p - 1)
        r = pow(g, k, p)
        k_inv = EL_GAMAL_ES._modular_inverse(k, p - 1)
        s = ((message - private_key * r) * k_inv) % (p - 1)
        print(f'GAMAL sign time: {time.time() - start}')
        return r, s
    
    @staticmethod
    def verify(message, r, s, p, g, public_key):
        start = time.time()
        v1 = pow(g, message, p)
        v2 = (pow(public_key, r, p) * pow(r, s, p)) % p
        print(f'GAMAL verify time: {time.time() - start}')
        return v1 == v2
    
    @staticmethod
    def _get_random_range(start, end):
        return math.floor(start + (end - start + 1) * random.random())
    
    @staticmethod
    def _modular_inverse(a, m):
        if math.gcd(a, m) != 1:
            raise ValueError("The modular inverse does not exist.")
        _, x, _ = EL_GAMAL_ES._extended_gcd(a, m)
        return x % m
    
    @staticmethod
    def _extended_gcd(a, b):
        if b == 0:
            return a, 1, 0
        d, x, y = EL_GAMAL_ES._extended_gcd(b, a % b)
        return d, y, x - (a // b) * y
    

p = 65537
g = 3
private_key, public_key = EL_GAMAL_ES.generate_keys(p, g)
message = 42
r, s = EL_GAMAL_ES.sign(message, p, g, private_key)
print(EL_GAMAL_ES.verify(message, r, s, p, g, public_key))

GAMAL sign time: 0.0
GAMAL verify time: 0.0
True


In [31]:
from typing import Tuple, Optional
Point = Tuple[int, int]
def tagged_hash(tag: str, msg: bytes) -> bytes:
    tag_hash = hashlib.sha256(tag.encode()).digest()
    return hashlib.sha256(tag_hash + tag_hash + msg).digest()
def x(P: Point) -> int:
    return P[0]
def y(P: Point) -> int:
    return P[1]
def point_add(P1: Optional[Point], P2: Optional[Point]) -> Optional[Point]:
    if P1 is None:
        return P2
    if P2 is None:
        return P1
    if (x(P1) == x(P2)) and (y(P1) != y(P2)):
        return None
    if P1 == P2:
        lam = (3 * x(P1) * x(P1) * pow(2 * y(P1), p - 2, p)) % p
    else:
        lam = ((y(P2) - y(P1)) * pow(x(P2) - x(P1), p - 2, p)) % p
    x3 = (lam * lam - x(P1) - x(P2)) % p
    return (x3, (lam * (x(P1) - x3) - y(P1)) % p)
def int_from_bytes(b: bytes) -> int:
    return int.from_bytes(b, byteorder="big")
def point_mul(P: Optional[Point], n: int) -> Optional[Point]:
    R = None
    for i in range(256):
        if (n >> i) & 1:
            R = point_add(R, P)
        P = point_add(P, P)
    return R
def bytes_from_int(x: int) -> bytes:
    return x.to_bytes(32, byteorder="big")
def bytes_from_point(P: Point) -> bytes:
    return bytes_from_int(x(P))
def xor_bytes(b0: bytes, b1: bytes) -> bytes:
    return bytes(x ^ y for (x, y) in zip(b0, b1))
def lift_x_square_y(b: bytes) -> Optional[Point]:
    x = int_from_bytes(b)
    if x >= p:
        return None
    y_sq = (pow(x, 3, p) + 7) % p
    y = pow(y_sq, (p + 1) // 4, p)
    if pow(y, 2, p) != y_sq:
        return None
    return (x, y)
def lift_x_even_y(b: bytes) -> Optional[Point]:
    P = lift_x_square_y(b)
    if P is None:
        return None
    else:
        return (x(P), y(P) if y(P) % 2 == 0 else p - y(P))
def int_from_bytes(b: bytes) -> int:
    return int.from_bytes(b, byteorder="big")
def hash_sha256(b: bytes) -> bytes:
    return hashlib.sha256(b).digest()
def is_square(x: int) -> bool:
    return int(pow(x, (p - 1) // 2, p)) == 1
def has_square_y(P: Optional[Point]) -> bool:
    infinity = is_infinity(P)
    if infinity: return False
    assert P is not None
    return is_square(y(P))
def has_even_y(P: Point) -> bool:
    return y(P) % 2 == 0
def pubkey_gen(seckey: bytes) -> bytes:
    d0 = int_from_bytes(seckey)
    if not (1 <= d0 <= n - 1):
        raise ValueError('The secret key must be an integer in the range 1..n-1.')
    P = point_mul(G, d0)
    assert P is not None
    return bytes_from_point(P)
def is_infinity(P: Optional[Point]) -> bool:
    return P is None

class SCH_ES:
    @staticmethod
    def generate_keys(p, q, g):
        private_key = SCH_ES._get_random_range(1, q - 1)
        public_key = pow(g, private_key, p)
        return private_key, public_key
    
    @staticmethod
    def sign(msg: bytes, seckey: bytes, aux_rand: bytes):
        start = time.time()
        if len(msg) != 32:
            raise ValueError('The message must be a 32-byte array.')
        d0 = int_from_bytes(seckey)
        if not (1 <= d0 <= n - 1):
            raise ValueError('The secret key must be an integer in the range 1..n-1.')
        if len(aux_rand) != 32:
            raise ValueError('aux_rand must be 32 bytes instead of %i.' % len(aux_rand))
        P = point_mul(G, d0)
        assert P is not None
        d = d0 if has_even_y(P) else n - d0
        t = xor_bytes(bytes_from_int(d), tagged_hash("BIP340/aux", aux_rand))
        k0 = int_from_bytes(tagged_hash("BIP340/nonce", t + bytes_from_point(P) + msg)) % n
        if k0 == 0:
            raise RuntimeError('Failure. This happens only with negligible probability.')
        R = point_mul(G, k0)
        assert R is not None
        k = n - k0 if not has_square_y(R) else k0
        e = int_from_bytes(tagged_hash("BIP340/challenge", bytes_from_point(R) + bytes_from_point(P) + msg)) % n
        sig = bytes_from_point(R) + bytes_from_int((k + e * d) % n)
        if not SCH_ES.verify(msg, bytes_from_point(P), sig):
            raise RuntimeError('The created signature does not pass verification.')
        print(f'SCHNORR sign time: {time.time() - start}')
        return sig
    
    @staticmethod
    def verify(msg: bytes, pubkey: bytes, sig: bytes):
        start = time.time()
        if len(msg) != 32:
            raise ValueError('The message must be a 32-byte array.')
        if len(pubkey) != 32:
            raise ValueError('The public key must be a 32-byte array.')
        if len(sig) != 64:
            raise ValueError('The signature must be a 64-byte array.')
        P = lift_x_even_y(pubkey)
        r = int_from_bytes(sig[0:32])
        s = int_from_bytes(sig[32:64])
        if (P is None) or (r >= p) or (s >= n):
            return False
        e = int_from_bytes(tagged_hash("BIP340/challenge", sig[0:32] + pubkey + msg)) % n
        R = point_add(point_mul(G, s), point_mul(P, n - e))
        if (R is None) or (not has_square_y(R)) or (x(R) != r):
            return False
        print(f'SCHNORR verify time: {time.time() - start}')
        return True
    
    @staticmethod
    def _get_random_range(start, end):
        return random.randint(start, end)
    
    @staticmethod
    def _hash_message(message):
        hash_object = hashlib.sha256(message.encode())
        hash_hex = hash_object.hexdigest()
        return hash_hex
    
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
G = (0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798, 
     0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8)

seckey1_hex = "B7E151628AED2A6ABF7158809CF4F3C762E7160F38B4DA56A784D9045190CFEF"
seckey2_hex = "B7E151628AED2A6ABF7158809CF4F3C762E7160F38B4DA56A784D9045190CFEE"
pubkey_hex="DFF1D77F2A671C5F36183726DB2341BE58FEAE1DA2DECED843240F7B502BA659"
pubkey2_hex = pubkey_gen(bytes.fromhex(seckey2_hex))
aux_rand_hex="0000000000000000000000000000000000000000000000000000000000000001"
msg_hex="243F6A8885A308D313198A2E03707344A4093822299F31D0082EFA98EC4E6C89"
sig_hex="0E12B8C520948A776753A96F21ABD7FDC2D7D0C0DDC90851BE17B04E75EF86A47EF0DA46C4DC4D0D1BCB8668C2CE16C54C7C23A6716EDE303AF86774917CF928"

msg = bytes.fromhex(msg_hex)
sig = bytes.fromhex(sig_hex)
seckey = bytes.fromhex(seckey1_hex)
pubkey = bytes.fromhex(pubkey_hex)
aux_rand = bytes.fromhex(aux_rand_hex)

sig_actual = SCH_ES.sign(msg, seckey, aux_rand)

print(sig == sig_actual)

SCHNORR verify time: 0.2536039352416992
SCHNORR sign time: 0.563579797744751
True


In [25]:
import subprocess
import json

lib_path = './Schnorr/bin/Debug/net8.0/Schnorr.exe'

class SCHNORR_RESERVE:
    @staticmethod
    def gen_keys():
        result = json.loads(subprocess.check_output([lib_path, 'gen']))
        return result['publicKey'], result['privateKey']
    
    @staticmethod
    def sign(message, private_key):
        result = subprocess.check_output([lib_path, 'sign', message, private_key])
        return json.dumps(json.loads((result)))
    
    @staticmethod
    def verify(message, sign, public_key):
        result = subprocess.check_output([lib_path, 'verify', message, sign, public_key])
        return int(result)

message = 'Qwerty'
public_key, private_key = SCHNORR_RESERVE.gen_keys()
sign = SCHNORR_RESERVE.sign(message, private_key)
print(SCHNORR_RESERVE.verify(message, sign, public_key))

0
